In [ ]:
from IPython.display import display, Markdown
from tbh.paths import REPO_ROOT_PATH
analysis_path = REPO_ROOT_PATH / "remote_cluster" / "outputs" / "48092744_age_spec_target" / "task_1"

In [ ]:
import tbh.plotting as pl
import pandas as pd
import arviz as az

intervention_scenarios = ["scenario_1", "scenario_2", "scenario_3"]
all_scenarios = ['baseline'] + intervention_scenarios
unc_dfs = {
    sc: pd.read_parquet(analysis_path / f"uncertainty_df_{sc}.parquet") for sc in all_scenarios
}
diff_outputs_dfs = {
    sc: pd.read_parquet(analysis_path / f"diff_quantiles_df_ref_baseline_{sc}.parquet") for sc in intervention_scenarios
}
idata = az.from_netcdf(analysis_path / "idata.nc")

In [ ]:
import yaml

with open(analysis_path / "details.yaml" , "r") as f:
    docs = list(yaml.safe_load_all(f))

model_config = docs[1]
analysis_config = docs[2]

In [ ]:
from tbh import runner_tools as rt
from tbh.model import get_tb_model
from estival.model import BayesianCompartmentalModel


params, priors, tv_params = rt.get_parameters_and_priors()

model = get_tb_model(model_config, tv_params)
bcm = BayesianCompartmentalModel(model, params, priors, rt.targets)

# Background and Introduction
## Aims and simulated population
This modelling analysis aims to explore and compare various scenarios of screening for TB and TBI in Kiribati. It focuses on simulating what the next phase of PEARL screening could potentially look like under different diagnostic approaches and considering different screening rates.
For this purpose, the simulated population is the population of South Tarawa excluding that already screened by the end of 2025. Thus, the model is designed to capture the part of South Tarawa extending from Nanikai to Bonriki. 
We will use the data collected so far through the PEARL study to estimate TB and TBI prevalence in the population already screened and assume that similar TB epidemiology applies to the population yet to be screened.

## Modelled scenarios


# Model structure and parameters

![Model structure. Not shown: age-stratification; natural mortality (all compartments); TB mortality (clinical TB compartments); self-recovery (subclinical TB compartments); reinfection from 'Contained', 'Cleared' and 'Recovered' compartments transitions back to 'Incipient'.](tb_model.png){#fig:tb_model width=100%}

# Model Calibration
## Model fits to data
The following figures show the model fits to calibration targets derived from data collected during the intervention. The black dots represent the observed quantities, whereas model estimates are shown in blue (solid line: median; dark shade: interquartile range; light shade: 95% CI).

In [ ]:
from matplotlib import pyplot as plt 
plt.style.use("ggplot")

selected_outputs = bcm.targets.keys()
for output in selected_outputs:
    _, ax = plt.subplots()
    pl.plot_model_fit_with_uncertainty(ax, unc_dfs['baseline'], output, bcm, x_lim=(2010, 2025))

## Parameters' posterior distributions
@fig-posteriors shows a comparison between the prior and posterior distributions of the calibrated model parameters. The prior distributions, shown in grey, reflect our knowledge about the parameter values **before** running the analysis and before considering the data used for calibration. Prior distributions are often chosen to be non-informative when there is very little evidence available to inform the parameter value. The posterior distributions, displayed in red, show the parameter values accepted during model calibration. These are the values found to be 'plausible', or appropriate to produce reasonable fits to the observations.

In [ ]:
#| label: fig-posteriors
#| fig-cap: "Prior and posterior distributions of calibrated parameters. Prior distributions are shown in grey, and posterior distributions in red."
pl.plot_post_prior_comparison(idata, analysis_config['burn_in'], req_vars=list(bcm.priors.keys()),
                              priors=list(bcm.priors.values()))

# Projected trajectories under various screening scenarios

# Estimated cumulative impact of interventions on TB disease and mortality

# Test integrating python var

In [ ]:
a = 125
display(Markdown(f"J'ai {a} flowers dans mon jardin"))